## Importing the Required Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import math
from datetime import datetime
import seaborn as sns
from sklearn.neighbors import DistanceMetric
from shapely.geometry import Point, Polygon
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from scipy.stats import mode

### Importing the Dataset Containing Physical Context Information From the BIM Model

In [2]:
df=pd.read_csv(r"G:\RTLS-BIM\Experiment8_zone_detection_pattern6\model_zone_detection5.csv")

#Changing the type of the numeric attributes to the float
Part1=df.drop(['family_type','comment'],axis=1).astype(float)
df=pd.concat([df[['family_type','comment']],Part1], axis=1)

### Importing the Dataset Containing the Target Estimated Locations

In [3]:
data =pd.read_csv(r"G:\RTLS-BIM\Experiment8_zone_detection_pattern6\Experiment8_zone_detection_pattern6.csv")

#Correcting the timestamp of the dataset

alk=pd.DataFrame(data['timestamp'].apply(lambda x: x.replace('@','')))
data['timestamp']=pd.DataFrame(pd.to_datetime(alk['timestamp'], infer_datetime_format=True))

# adding the z-axes to the locations

data['location_Z']=0

data=data.dropna().drop_duplicates(subset=['timestamp']).sort_values('timestamp').reset_index(drop=True)

#### Generating a dataset in which the number of records (locations) multiplied by the number of elements and the dataset of the element (x,y) coordinates is repeated by the number of the records. 

In [4]:
Locations=pd.DataFrame(np.repeat(data.values, len(df.index), axis=0), columns=data.columns)
Geo=pd.concat([df]*len(data.index))
Geo=Geo.reset_index(drop=True)
Final=pd.concat([Locations,Geo], axis=1)

#### Checking the zone where the target is located

In [5]:
acx1=list(Final[['X1','Y1']].to_records(index=False))
acx2=list(Final[['X2','Y2']].to_records(index=False))
acx3=list(Final[['X3','Y3']].to_records(index=False))
acx4=list(Final[['X4','Y4']].to_records(index=False))
cxx=Final['location_X']
cxy=Final['location_Y']

rows5 = []
for (a,b) in zip(zip(acx1,acx2,acx3,acx4),zip(cxx,cxy)):
        poly = (Polygon(a))
        p1 = Point((b))
        polyy=poly.contains(p1)
        rows5.append(polyy)
    
Final['surrounded']=pd.DataFrame(rows5)

In [6]:
Final=Final.iloc[Final.groupby(Final.index // len(df.index)).surrounded.idxmax()].reset_index()

### Here is an algorithm to calculate the distance of a target point from a segment (of an element) and to detrmine the (x,y) coordiantes of the nearest point on the segment to the target point

In [7]:
def dot(v,w):
    x,y,z = v
    X,Y,Z = w
    return x*X + y*Y + z*Z

def length(v):
    x,y,z = v
    return math.sqrt(x*x + y*y + z*z)

def vector(b,e):
    x,y,z = b
    X,Y,Z = e
    return (X-x, Y-y, Z-z)

def unit(v):
    x,y,z = v
    mag = length(v)
    return (x/mag, y/mag, z/mag)

def distance(p0,p1):
    return length(vector(p0,p1))

def scale(v,sc):
    x,y,z = v
    return (x * sc, y * sc, z * sc)

def add(v,w):
    x,y,z = v
    X,Y,Z = w
    return (x+X, y+Y, z+Z)

def pnt2line(pnt, start, end):
    line_vec = vector(start, end)
    pnt_vec = vector(start, pnt)
    line_len = length(line_vec)
    line_unitvec = unit(line_vec)
    pnt_vec_scaled = scale(pnt_vec, 1.0/line_len)
    t = dot(line_unitvec, pnt_vec_scaled)    
    if t < 0.0:
        t = 0.0
    elif t > 1.0:
        t = 1.0
    nearest = scale(line_vec, t)
    dist = distance(nearest, pnt_vec)
    nearest = add(nearest, start)
    return (dist, nearest)

### Applying the algorithm for the segments of each element 

In [8]:
pnt1=list(Final[['location_X','location_Y','location_Z']].to_records(index=False))
start1=list(Final[['X1','Y1','Z1']].to_records(index=False))
end1=list(Final[['X2','Y2','Z2']].to_records(index=False))

pnt2=list(Final[['location_X','location_Y','location_Z']].to_records(index=False))
start2=list(Final[['X2','Y2','Z2']].to_records(index=False))
end2=list(Final[['X3','Y3','Z3']].to_records(index=False))

pnt3=list(Final[['location_X','location_Y','location_Z']].to_records(index=False))
start3=list(Final[['X3','Y3','Z3']].to_records(index=False))
end3=list(Final[['X4','Y4','Z4']].to_records(index=False))

pnt4=list(Final[['location_X','location_Y','location_Z']].to_records(index=False))
start4=list(Final[['X4','Y4','Z4']].to_records(index=False))
end4=list(Final[['X1','Y1','Z1']].to_records(index=False))

In [9]:
rows1 = []
for i, b, s in zip(pnt1,start1,end1):
    m=(pnt2line(i,b,s))
    rows1.append(m)
    
Intersect1=pd.DataFrame(rows1).rename({0:'distance1',1:'intersection_coordinates1'},axis=1)

rows2 = []
for i, b, s in zip(pnt2,start2,end2):
    m=(pnt2line(i,b,s))
    rows2.append(m)

Intersect2=pd.DataFrame(rows2).rename({0:'distance2',1:'intersection_coordinates2'},axis=1)

rows3 = []
for i, b, s in zip(pnt3,start3,end3):
    m=(pnt2line(i,b,s))
    rows3.append(m)
    
Intersect3=pd.DataFrame(rows3).rename({0:'distance3',1:'intersection_coordinates3'},axis=1)

rows4 = []
for i, b, s in zip(pnt4,start4,end4):
    m=(pnt2line(i,b,s))
    rows4.append(m)

Intersect4=pd.DataFrame(rows4).rename({0:'distance4',1:'intersection_coordinates4'},axis=1)

Merged=pd.concat([Final,Intersect1,Intersect2,Intersect3,Intersect4],axis=1)

####  The time difference between two executive records is calculated to find the time spent on the workspaces by the target

In [10]:
Final['diff_seconds'] = pd.DataFrame(Final['timestamp'].diff(1))['timestamp'].dt.total_seconds()
Final=Final[['timestamp', 'family_type', 'comment',
       'element_id', 'surrounded', 'diff_seconds','experiment']]

In [11]:
# To remove noises and downtime of the BLE beacons worn by the workers, the records generated after 6 seconds should be removed.
Final=Final[(Final['diff_seconds']<=5)].sort_values('timestamp').reset_index().drop('index',axis=1)
Final['comment'].fillna('safe', inplace=True)

### Actual Time Spent and Records Frequency in the Workspaces

In [13]:
time_spent = Final.groupby('comment').sum()
records_number = pd.DataFrame(Final.groupby('comment').count()['timestamp'])
final_dataset = pd.merge(time_spent, records_number, left_index=True, right_index=True)
final_dataset = final_dataset.drop(['element_id','surrounded'],axis=1)
final_dataset.rename({'diff_seconds':'time_spent','timestamp':'records_number'},axis=1)

,time_spent,records_number
comment,,
safe,2.0,1
workspace_A,209.0,113
workspace_B,172.0,91
workspace_C,263.0,136
workspace_D,165.0,92
workspace_E,203.0,109
